In [43]:
import cv2
import os
from random import *
import numpy as np
import math
import glob
from numba import jit, prange
import time
import warnings
from numba import cuda
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from numba.typed import List
from argparse import ArgumentParser
parser = ArgumentParser()

@jit(nopython=True)
def Random_Centroids(k, image, height, width):
    Centroids_R,Centroids_G, Centroids_B = np.array([1] * k), np.array([1] * k), np.array([1] * k)
    
    for i in range(k):
        Centroids_R[i] = image[randrange(width)][randrange(height)][0]
        Centroids_G[i] = image[randrange(width)][randrange(height)][1]
        Centroids_B[i] = image[randrange(width)][randrange(height)][2]
    
    return Centroids_R, Centroids_G, Centroids_B

@cuda.jit
def Choose_Centroid(Index_Map, Map_Centroids, image, Centroids):
  
    row, col = cuda.grid(2)
    
    if row < image.shape[0] and col < image.shape[1]:
        pixel = image[row][col]
        index = 0
        temp = 100*len(Centroids)

        for i in range(len(Centroids)):
            distance = (math.sqrt((math.pow(int(Centroids[i][0])-int(pixel[0]),2))+(math.pow(int(Centroids[i][1])-int(pixel[1]),2))+(math.pow(int(Centroids[i][2])-int(pixel[2]),2))))
            if (distance < temp):
                temp = distance
                index = i
        
        Index_Map[row][col][0] = index
        Map_Centroids[row][col][0] = Centroids[index][0]
        Map_Centroids[row][col][1] = Centroids[index][1]
        Map_Centroids[row][col][2] = Centroids[index][2]
        
@cuda.jit
def ReChoose_Centroid(index_map, image, k, Centroids_):
    pos = cuda.grid(1)
    
    for row in range(image.shape[0]):
        for col in range(image.shape[1]):
            temp = index_map[row][col]
            
            if (pos < 3):
                Centroids_[pos][temp[0]] += image[row][col][pos]
            if (pos < 6 and pos >= 3):
                Centroids_[pos][temp[0]] += 1
       
    for i in range(k):
        print(Centroids_[0][i], Centroids_[3][i])
        Centroids_[0][i] = float(Centroids_[0][i]/Centroids_[3][i])
        Centroids_[1][i] = float(Centroids_[1][i]/Centroids_[4][i])
        Centroids_[2][i] = float(Centroids_[2][i]/Centroids_[5][i])
        

def SegmentationImage(image, k):
    
    timing_choose = 0
    timing_rechoose = 0
    img = cv2.imread(image)
    height, width, channels = img.shape

    Centroids_R, Centroids_G, Centroids_B = Random_Centroids(k, img, width, height)

    #First Centroids
    Centroids = np.array([[1, 2, 3]] * k, np.int64)
    
    for i in range(k):
        Centroids[i] = [Centroids_R[i], Centroids_G[i], Centroids_B[i]]

    #KMeans
    for i in range(10):
        
        image_global_mem = cuda.to_device(img)
        Centroids_global_mem = cuda.to_device(Centroids)

        Map_Centroids_global_mem = cuda.device_array((height, width, 3))
        Index_Map_global_mem = cuda.device_array((height, width, 1))

        threadsperblock = (32,32)
        blockspergrid_x = int(math.ceil(height / threadsperblock[0]))
        blockspergrid_y = int(math.ceil(width / threadsperblock[1]))
        blockspergrid = (blockspergrid_x, blockspergrid_y)
        
        start_time1 = time.time()
        Choose_Centroid[blockspergrid, threadsperblock](Index_Map_global_mem, Map_Centroids_global_mem, image_global_mem, Centroids_global_mem)
        timing_choose = timing_choose + (time.time() - start_time1)
        
        Index_Map = Index_Map_global_mem.copy_to_host().astype(int) 
    
        start_time2 = time.time()
        Index_Map_global_mem = cuda.to_device(Index_Map)
        
        Centroids_R,Centroids_G, Centroids_B = np.array([0] * k), np.array([0] * k), np.array([0] * k)
        n_Centroids_R, n_Centroids_G, n_Centroids_B = np.array([0] * k), np.array([0] * k), np.array([0] * k)
        
        Centroids_ = [Centroids_R,Centroids_G, Centroids_B, n_Centroids_R, n_Centroids_G, n_Centroids_B]
        Centroids__global_mem = cuda.to_device(Centroids_)
#         print(Centroids)  
        ReChoose_Centroid[1, 6](Index_Map_global_mem, image_global_mem, k, Centroids__global_mem)
        Centroids_ = Centroids__global_mem.copy_to_host()    
        timing_rechoose = timing_rechoose + (time.time() - start_time2)
        
        
        for i in range(k):
            Centroids[i] = [Centroids_[0][i], Centroids_[1][i], Centroids_[2][i]]
#         print(Centroids)
        
        Map_Centroids = Map_Centroids_global_mem.copy_to_host().astype(int)
        
    print("--- Choose Time: %s seconds ---" % timing_choose)
    print("--- ReChoose Time: %s seconds ---" % timing_rechoose)
    return Map_Centroids
    

In [44]:
start_time = time.time()
out_img = SegmentationImage("images.jpg", 3)
print("--- %s seconds ---" % (time.time() - start_time))
cv2.imwrite('kmeans_parallel_test.jpg', out_img)

--- Choose Time: 0.43062376976013184 seconds ---
--- ReChoose Time: 11.119109392166138 seconds ---
--- 13.592296600341797 seconds ---


True

In [ ]:
k = 10
SSE = [0] * k 

for i in range(k):
    start_time = time.time()
    SSE[i] = 0
    
    out_img = SegmentationImage('images.jpg',i + 1)
    img = cv2.imread('images.jpg')
    
    for x in range(len(img)):
        for y in range(len(img[0])):
            SSE[i] = SSE[i] + math.sqrt(math.pow((int(img[x][y][0])-int(out_img[x][y][0])),2) + math.pow((int(img[x][y][1])-int(out_img[x][y][1])),2) + math.pow((int(img[x][y][2])-int(out_img[x][y][2])),2))
    
    print("--- %s seconds ---" % (time.time() - start_time))
    
ypoints = np.array(SSE)
plt.plot(ypoints, linestyle = 'dotted')
plt.show()
temp = []

print(SSE)

for i in range(len(SSE) - 3):
    temp.append(abs(int(SSE[i] - SSE[i+1]) - int(SSE[i+1] - SSE[i+2]) - int(SSE[i+2] - SSE[i+3])))

print(temp)
k = temp.index(min(temp)) + 1
print("---- Number of Cluster: %s ------" %k)
# start_time = time.time()

# count = 1
# images = glob.glob("D:\Work\LTSSUD\*.jpg")
# for i in images:
#     out_img = SegmentationImage(i,k)
#     cv2.imwrite(i.replace('frame','landscape'), out_img)
#     print("Complete {}/{}".format(count, len(images)))
#     count = count + 1

# print("--- %s seconds ---" % (time.time() - start_time))

In [11]:
import numpy as np
import cv2 as cv

img = cv.imread('images.jpg')
Z = img.reshape((-1,3))
# convert to np.float32
Z = np.float32(Z)
# define criteria, number of clusters(K) and apply kmeans()
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 3
ret,label,center=cv.kmeans(Z,K,None,criteria,10,cv.KMEANS_RANDOM_CENTERS)

center = np.uint8(center)
res = center[label.flatten()]
res2 = res.reshape((img.shape))

cv2.imwrite('kmeans_opencv.jpg', res2)

True